# Creating your own pdf

A fundamental design choice of zfit is the ability to create custom pdfs and functions in an easy way.

There are two ways, an easier for most use-cases and an advanced way of creating your own pdf.

## The simple way

While the same works for functions, an example with a PDF is shown here

In [1]:
import zfit
from zfit import ztf

In [2]:
class MyGauss(zfit.pdf.ZPDF):
    _N_OBS = 1  # dimension, can be omitted
    _PARAMS = ['mean', 'std']  # the name of the parameters
    
    def _unnormalized_pdf(self, x):
        x = ztf.unstack_x(x)  # returns a list with the columns: do x, y, z = ztf.unstack_x(x) for 3D
        mean = self.params['mean']
        std = self.params['std']
        return ztf.exp(- ((x - mean)/std)**2)
        

Done

In [3]:
data_np = np.random.random(size=1000)
data = zfit.data.Data.from_numpy(array=data_np, obs='obs1')

Create two parameters and an instance of your own pdf

In [4]:
mean = zfit.Parameter("mean", 1.)
std = zfit.Parameter("std", 1.)
my_gauss = MyGauss(obs='obs1', mean=mean, std=std)

In [5]:
probs = my_gauss.pdf(data, norm_range=(-3, 4))

Instructions for updating:
keep_dims is deprecated, use keepdims instead


In [6]:
probs_np = zfit.run(probs)
print(probs_np[:20])

[0.3750314  0.45359967 0.36579647 0.38784419 0.54378104 0.55869691
 0.46385139 0.24879392 0.35822478 0.38186877 0.26458103 0.28732484
 0.54404543 0.49188169 0.35697661 0.48529191 0.28756458 0.38787331
 0.24605188 0.41959645]


We could improve our PDF by registering an integral

In [7]:
def gauss_integral_from_any_to_any(limits, params, model):
    (lower,), (upper,) = limits.limits
    mean = params['mean']
    std = params['std']
    # write your integral here
    return 42.  # dummy integral

In [8]:
limits = zfit.Space.from_axes(axes=0, limits=(zfit.Space.ANY_LOWER, zfit.Space.ANY_UPPER))
MyGauss.register_analytic_integral(func=gauss_integral_from_any_to_any, limits=limits)

## Advanced Custom PDF

Subclass BasePDF. The `_unnormalized_pdf` has to be overriden and, in addition, the `__init__`.

Any of the public main methods (`pdf`, `integrate`, `partial_integrate` etc.) can **always** be overriden by implementing the function with a leading underscore, e.g. implement `_pdf` to directly controls `pdf`. In case, during execution of your own method, it is found to be a bad idea to have overridden the default methods, throwing a `NotImplementedError` will restore the default behavior.

In [9]:
TOBEDONE

NameError: name 'TOBEDONE' is not defined